In [10]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_core.runnables import RunnablePassthrough
from rouge import Rouge
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from sentence_transformers import SentenceTransformer, util
import numpy as np


In [11]:
local_path = r"D:\GitHub\Projetos\Mestrado\EnergyContext\pdf\appendixa_0.pdf"
if local_path:
    loader = UnstructuredPDFLoader(file_path=local_path)
    data = loader.load()
else:
    raise FileNotFoundError("Upload a PDF file")


In [12]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=100)
chunks = text_splitter.split_documents(data)


In [13]:
vector_db = Chroma.from_documents(
    documents=chunks, 
    embedding=OllamaEmbeddings(model="nomic-embed-text", show_progress=True),
    collection_name="local-rag"
)


OllamaEmbeddings: 100%|██████████| 3/3 [00:08<00:00,  2.70s/it]


In [14]:
'''
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an electrical engineer. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database, and only the database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide an answer combining the most important points of these five versions.
    Original question: {question}"""
)
'''
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an electrical engineer. Your task is to generate a response based on the vector database.Your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide an answer as close as possible to what is in the document.
    Original question: {question}"""
)


In [15]:
local_model = "mistral"
llm = ChatOllama(model=local_model)
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)


In [16]:
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)


In [17]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


In [18]:
chain.invoke("What are technologies used to provide indoor environmental comfort?")

OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.08s/it]


' The document provided does not explicitly mention specific technologies used to provide indoor environmental comfort, but it does mention Combined Heat and Power (CHP) systems as an efficient approach for generating power and thermal energy from a single fuel source, which can help maintain indoor comfort by providing both heating and cooling. However, for more detailed information about specific HVAC (Heating, Ventilating, and Air Conditioning) technologies used for indoor environmental control, one might need to refer to specialized resources or literature on HVAC systems and building design.'

In [19]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from sentence_transformers import SentenceTransformer, util
import numpy as np
import pandas as pd

# Define a set of handwritten test queries and expected answers
test_queries = [
    "What is the relationship between Megawatt (MW) and Megawatt-hour (MWh)?",
    "Who prepared the Annual Energy Outlook?",
    "What is the Clean Air Act?",
    "What is an Electric Generating Unit?",
    "How Electricity Dispatch Models optimize the dispatch of a system?",
    "Who maintains the Emissions & Generation Resource Integrated Database?",
    "Give me examples of Energy Efficiency",
    "What are ISOs?",
    "What are Power Pools?",
    "What does the Public Utilities Commission or Public Service Commission do?",
    "What does Electricity Dispatch Models simulate?",
]

expected_answers = [
    "A megawatt (MW) is a unit of power representing the rate at which energy is used or generated, while a megawatt-hour (MWh) is a unit of energy representing the total amount of energy used or generated over an hour.",
    "The U.S. Department of Energy’s Energy Information Administration (EIA)",
    "The Clean Air Act (CAA) is the law that defines the U.S. Environmental Protection Agency’s responsibilities for protecting and improving the nation's air quality and the stratospheric ozone layer",
    "An entity that supplies electricity to the electricity system relying on a variety of fuels.",
    "These models optimize the dispatch of the system based on the variable costs of each resource and any operational constraints that have been entered into the model.",
    "The U.S. Environmental Protection Agency",
    "High-efficiency appliances; efficient lighting; high-efficiency heating, ventilating and air conditioning systems or control modifications; efficient building design; advanced electric motor drives; combined heat and power; and heat recovery systems.",
    "Independent System Operators",
    "A power pool is an association of two or more interconnected electric systems that agree to coordinate operations and planning for improved reliability and efficiencies.",
    "Regulates the rates and services of a public utility.",
    "Simulate the dynamic operation of the electric system, generally on a least-cost system dispatch",
]

# Function to categorize scores into low, mid, and high
def categorize_score(score, thresholds):
    if score < thresholds['low']:
        return 'low'
    elif score < thresholds['mid']:
        return 'mid'
    else:
        return 'high'

# Function to evaluate the RAG system using BLEU, ROUGE, and semantic similarity
def evaluate_rag_system(chain, test_queries, expected_answers):
    # Get predictions from the RAG system
    predictions = [chain.invoke(query) for query in test_queries]
    print("Predictions:", predictions)  # Debug: Print predictions to see what the system returns

    # Initialize metrics
    bleu_scores = []
    rouge_scores = []
    semantic_similarities = []

    # Initialize ROUGE scorer
    rouge = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    
    # Load a sentence transformer model for semantic similarity
    model = SentenceTransformer('all-MiniLM-L6-v2')
    
    for pred, ref in zip(predictions, expected_answers):
        # Calculate BLEU score
        bleu_score = sentence_bleu([ref.split()], pred.split(), smoothing_function=SmoothingFunction().method1)
        bleu_scores.append(bleu_score)
        
        # Calculate ROUGE scores
        rouge_score = rouge.score(ref, pred)
        rouge_scores.append(rouge_score)
        
        # Calculate semantic similarity
        pred_embedding = model.encode(pred, convert_to_tensor=True)
        ref_embedding = model.encode(ref, convert_to_tensor=True)
        semantic_similarity = util.pytorch_cos_sim(pred_embedding, ref_embedding).item()
        semantic_similarities.append(semantic_similarity)
    
    # Calculate average scores
    avg_bleu = np.mean(bleu_scores)
    avg_rouge1 = np.mean([score['rouge1'].fmeasure for score in rouge_scores])
    avg_rouge2 = np.mean([score['rouge2'].fmeasure for score in rouge_scores])
    avg_rougeL = np.mean([score['rougeL'].fmeasure for score in rouge_scores])
    avg_semantic_similarity = np.mean(semantic_similarities)

    # Define thresholds for categorizing scores
    thresholds = {
        'bleu': {'low': 0.2, 'mid': 0.5},
        'rouge1': {'low': 0.2, 'mid': 0.5},
        'rouge2': {'low': 0.1, 'mid': 0.3},
        'rougeL': {'low': 0.2, 'mid': 0.5},
        'semantic': {'low': 0.5, 'mid': 0.7}
    }

    # Categorize the average scores
    bleu_category = categorize_score(avg_bleu, thresholds['bleu'])
    rouge1_category = categorize_score(avg_rouge1, thresholds['rouge1'])
    rouge2_category = categorize_score(avg_rouge2, thresholds['rouge2'])
    rougeL_category = categorize_score(avg_rougeL, thresholds['rougeL'])
    semantic_category = categorize_score(avg_semantic_similarity, thresholds['semantic'])

    # Print results
    print(f"Average BLEU Score: {avg_bleu:.2f} ({bleu_category})")
    print(f"Average ROUGE-1 Score: {avg_rouge1:.2f} ({rouge1_category})")
    print(f"Average ROUGE-2 Score: {avg_rouge2:.2f} ({rouge2_category})")
    print(f"Average ROUGE-L Score: {avg_rougeL:.2f} ({rougeL_category})")
    print(f"Average Semantic Similarity: {avg_semantic_similarity:.2f} ({semantic_category})")

    return {
        'avg_bleu': avg_bleu,
        'avg_rouge1': avg_rouge1,
        'avg_rouge2': avg_rouge2,
        'avg_rougeL': avg_rougeL,
        'avg_semantic_similarity': avg_semantic_similarity,
        'categories': {
            'bleu': bleu_category,
            'rouge1': rouge1_category,
            'rouge2': rouge2_category,
            'rougeL': rougeL_category,
            'semantic': semantic_category
        },
        'predictions': predictions  # Return the predictions
    }

# Assuming 'chain' is your RAG system already defined
# For example, chain.invoke("Your query") should return the answer from the RAG system
evaluation_results = evaluate_rag_system(chain, test_queries, expected_answers)

# Create a DataFrame with the results
df = pd.DataFrame({
    'Query': test_queries,
    'Expected Answer': expected_answers,
    'Predicted Answer': evaluation_results['predictions']
})

# Save the DataFrame to an Excel file
df.to_excel('evaluation_results.xlsx', index=False)

"""
# Print the answers that the model returns
for query, prediction in zip(test_queries, evaluation_results['predictions']):
    print(f"Query: {query}")
    print(f"Prediction: {prediction}")"""


OllamaEmbeddings: 100%|██████████| 1/1 [00:03<00:00,  3.73s/it]


Predictions: [' Megawatt (MW) is a measure of power, or rate of energy flow, typically used for electric power in watts (W). On the other hand, Megawatt-hour (MWh) is a measure of energy, representing the amount of energy delivered over one hour at a rate of one megawatt. So, 1 MWh is equal to the energy supplied by 1 MW operating for one hour.', ' The Annual Energy Outlook is prepared by the U.S. Department of Energy’s Energy Information Administration (EIA).', " The Clean Air Act (CAA) is a U.S. law that defines the Environmental Protection Agency's responsibilities for protecting and improving the nation's air quality and the stratospheric ozone layer. It was last significantly amended in 1990, with minor changes made since then. The CAA sets National Ambient Air Quality Standards for six common air pollutants known as criteria air pollutants, which include particle pollution, ground-level ozone, carbon monoxide, sulfur oxides, nitrogen oxides, and lead.", ' An Electric Generating U

d:\ProgramData\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Average BLEU Score: 0.14 (low)
Average ROUGE-1 Score: 0.39 (mid)
Average ROUGE-2 Score: 0.28 (mid)
Average ROUGE-L Score: 0.35 (mid)
Average Semantic Similarity: 0.72 (high)


'\n# Print the answers that the model returns\nfor query, prediction in zip(test_queries, evaluation_results[\'predictions\']):\n    print(f"Query: {query}")\n    print(f"Prediction: {prediction}")'